<a href="https://colab.research.google.com/github/Fathimath-Rifna-VK/fmml2021/blob/main/Module_5_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
N_samples = 400
x_1 = np.random.uniform(low= - 4.0, high = 4.0, size=N_samples)
x_2 = np.random.uniform(low = -4.0, high = 4.0, size=N_samples)
noise = np.random.normal(0, 0.01, N_samples)
y = 2 * x_1 + 3 * x_2 + noise

In [ ]:
a = np.linspace(-10, 10, 200)
b = np.linspace(-10, 10, 200)
# a, b are the parameter values

X, Y = np.meshgrid(a, b)
J = np.ones(shape = (len(a), len(b)))

# compute J for all pairs of a, b
for i in range(len(a)):
    for j in range(len(b)):
        J[i][j] = np.linalg.norm((2 * x_1 + 3 * x_2) - (a[i] * x_1 + b[j] * x_2)) / N_samples

# creating the 3D Plot
fig = plt.figure(figsize=(10, 10))
sub = fig.add_subplot(111, projection='3d')
sub.set_title('Relationship between J and (a, b)')
sub.set_xlabel('a')
sub.set_ylabel('b')
sub.set_zlabel('J')
sub.plot_surface(X, Y, J, cmap = cm.coolwarm)
plt.show()

In [ ]:
# calculating the optimal learning rate:

Hessian = 2 * np.asarray([[np.dot(x_1, x_1), np.dot(x_1, x_2)], [np.dot(x_1, x_2), np.dot(x_2, x_2)]])
eigvals, _ = np.linalg.eig(Hessian)
eta_opt = 1 / max(eigvals)
print("Optimal Learning Rate:", str(eta_opt))

In [ ]:
def loss(a, b):
    return np.linalg.norm((a * x_1 + b * x_2) - y) / N_samples

def gradient_descent(eta_multiplier):
    global Hessian
    a = b = 0
    errors = [loss(a, b)]
    seq = [(0, 0)]
    MAX_ITER = 100
    iteration = 0

    while errors[-1] >= 1e-3 and iteration < MAX_ITER:
        # using the accurate version of optimal learning rate
        del_J_del_a = 2 * a * np.dot(x_1, x_1) + 2 * b * np.dot(x_1, x_2) - 2 * np.dot(y, x_1)
        del_J_del_b = 2 * a * np.dot(x_1, x_2) + 2 * b * np.dot(x_2, x_2) - 2 * np.dot(y, x_2)
        del_J = np.asarray([del_J_del_a, del_J_del_b])
        if np.matmul(np.matmul(del_J.T, Hessian), del_J) == 0:
            # indicates optimum has been reached
            break
            
        eta_opt = np.dot(del_J, del_J) / np.matmul(np.matmul(del_J.T, Hessian), del_J)
        
        eta = eta_opt * eta_multiplier

        # update step
        a = a - eta * del_J_del_a
        b = b - eta * del_J_del_b
        seq.append((a, b))
        errors.append(loss(a, b))
        iteration += 1
    
    return errors, seq, a, b

In [ ]:
def make_plot(seq, errors):
    make_error_plot(errors)
    make_convergence_plot(seq)

def make_error_plot(errors):
    fig = plt.figure(figsize=(7, 7))
    p = fig.add_subplot('111')
    p.set_title('Error v/s Epoch')
    p.set_xlabel('Epoch Number')
    p.set_ylabel('Error')
    p.plot(list(range(1, len(errors))), errors[1:])
    plt.show()

def make_convergence_plot(seq, problem = 1, xlabel = 'a', ylabel = 'b'):
    # problem can be 1, 2, 3
    fig = plt.figure(figsize=(7, 7))
    q = fig.add_subplot('111')

    a_min = min([s[0] for s in seq])
    a_max = max([s[0] for s in seq])

    b_min = min([s[1] for s in seq])
    b_max = max([s[1] for s in seq])

    eps = 0.95
    if xlabel == 'a':
        eps = 0.2

    if problem == 1:
        a = np.linspace(a_min - eps, a_max + eps, 500)
        b = np.linspace(b_min - eps, b_max + eps, 500)
    
    else:
        a = np.linspace(-1, 1, 500)
        b = np.linspace(-1, 1, 500)

    X, Y = np.meshgrid(a, b)
    J = np.ones(shape = (len(a), len(b)))

    # compute J for all pairs of a, b
    for i in range(len(a)):
        for j in range(len(b)):
            if problem == 1:
                J[i][j] = np.linalg.norm((2 * x_1 + 3 * x_2) - (a[i] * x_1 + b[j] * x_2))
            elif problem == 2:
                J[i][j] = a[i] * a[i] + 100 * np.power((b[j] - a[i] * a[i]), 2) 
            elif problem == 3:
                J[i][j] = 50 / 9 * np.power((a[i] * a[i] + b[j] * b[j]), 3) - 209 / 18 * np.power((a[i] * a[i] + b[j] * b[j]), 2) + 59 / 9 * (a[i] * a[i] + b[j] * b[j])
    
    q.set_title('Convergence Path')
    q.set_xlabel(xlabel)
    q.set_ylabel(ylabel)
    q.contour(X, Y, J, cmap = cm.coolwarm)

    # plotting the arrow-ed path
    aspace = 0.1 # scaling factor

    # r is the distance spanned between pairs of points
    r = [0]
    for i in range(1,len(seq)):
        dx = seq[i][0]-seq[i-1][0]
        dy = seq[i][1]-seq[i-1][1]
        val = np.sqrt(dx ** 2 + dy ** 2)
        r.append(val)
    r = np.asarray(r)

    # r_sum is the cumulative sum of r
    r_sum = []
    for i in range(len(r)):
        r_sum.append(r[0: i].sum())
    r_sum.append(r.sum())

    arrow_data = [] # holds tuples of (x, y, theta) for each arrow
    arrow_pos = 0 # current point on walk along data
    r_count = 1

    while arrow_pos < r.sum():
        x1, x2 = seq[r_count-1][0], seq[r_count][0]
        y1, y2 = seq[r_count-1][1], seq[r_count][1]
        da = arrow_pos - r_sum[r_count] 
        theta = np.arctan2((x2 - x1), (y2 - y1))
        ax = np.sin(theta) * da + x1
        ay = np.cos(theta) * da + y1

        arrow_data.append((ax, ay, theta))
        arrow_pos += aspace

        while arrow_pos > r_sum[r_count + 1]: 
            r_count += 1
            if arrow_pos > r_sum[-1]:
                break

    for ax, ay, theta in arrow_data:
        q.arrow(ax, ay, np.sin(theta) * aspace / 10, np.cos(theta) * aspace / 10, head_width = aspace / 2.5 , head_length = aspace / 1.5 , color = 'darkgreen')

    q.plot([s[0] for s in seq], [s[1] for s in seq], color = 'green')
    

    plt.show()

In [ ]:
# eta = (0.45) * eta_opt
e, s, a_opt, b_opt = gradient_descent((0.45))
make_plot(s, e)
# output:
print("Optimal Values for: ")
print("a : " + str(a_opt))
print("b : " + str(b_opt))

In [ ]:
# eta = eta_opt
e, s, a_opt, b_opt = gradient_descent(1)
make_plot(s, e)
# output:
print("Optimal Values for: ")
print("a : " + str(a_opt))
print("b : " + str(b_opt))

In [ ]:
# eta = (1.5) * eta_opt
e, s, a_opt, b_opt = gradient_descent(1.5)
make_plot(s, e)
# output:
print("Optimal Values for: ")
print("a : " + str(a_opt))
print("b : " + str(b_opt))

In [ ]:
def check_convergence(seq):
    threshold = 1e-8
    if len(seq) > 1 and np.abs(seq[-1][0] - seq[-2][0]) < threshold and np.abs(seq[-1][1] - seq[-2][1]) < threshold:
        return True
    return False

In [ ]:
def plain_gradient_descent(f, grad_x, grad_y):
    # f is the function to be minimized
    # grad_x and grad_y are functions which accept (x, y)
    # and return partial derivatives of f w.r.t x and y respectively
    
    x = np.random.random()
    y = np.random.random()
    # initialized values

    seq = [(x, y)] # sequence of x, y
    MAX_ITER = 1e5
    iteration = 0
    eta = 1e-3

    while iteration < MAX_ITER and check_convergence(seq) == False:
        # update step
        del_x = grad_x(x, y)
        del_y = grad_y(x, y)
        x = x - eta * del_x
        y = y - eta * del_y
        
        seq.append((x, y))
        iteration += 1
    
    print("Number of Iterations: " + str(iteration))
    print("f* = " + str(f(x, y)))
    print("(x*, y*) = " + "(" + str(seq[-1][0]) + ", " + str(seq[-1][1]) + ")")
    return seq

In [ ]:
def nesterov_gradient_descent(f, grad_x, grad_y):
    # f is the function to be minimized
    # grad_x and grad_y are functions which accept (x, y)
    # and return partial derivatives of f w.r.t x and y respectively
    
    x = np.random.random()
    y = np.random.random()
    V_x = [0]
    V_y = [0]
    # initialized values

    seq = [(x, y)] # sequence of x, y
    MAX_ITER = 1e5
    iteration = 0
    alpha = 1e-4
    beta = 0.9

    while iteration < MAX_ITER and check_convergence(seq) == False:
        # update step
        x_star = x - alpha * V_x[-1]
        y_star = y - alpha * V_y[-1]
        
        del_x = grad_x(x_star, y_star)
        del_y = grad_y(x_star, y_star)

        V_x.append(beta * V_x[-1] + (1 - beta) * del_x)
        V_y.append(beta * V_y[-1] + (1 - beta) * del_y)
                   
        x = x - alpha * V_x[-1]
        y = y - alpha * V_y[-1]
        
        seq.append((x, y))
        iteration += 1

    print("Number of Iterations: " + str(iteration))
    print("f* = " + str(f(x, y)))
    print("(x*, y*) = " + "(" + str(seq[-1][0]) + ", " + str(seq[-1][1]) + ")")
    return seq

In [ ]:
seq_plain = plain_gradient_descent(lambda x, y: x ** 2 + 100 * ((y - x ** 2) ** 2),lambda x, y: (2 * x * (1 - 200 * (y - x ** 2))),  lambda x, y : (200 * (y - x ** 2)))
make_convergence_plot(seq_plain, 2, 'x', 'y')

In [ ]:
seq_nesterov = nesterov_gradient_descent(lambda x, y: x ** 2 + 100 * ((y - x ** 2) ** 2), lambda x, y: (2 * x * (1 - 200 * (y - x ** 2))),  lambda x, y : (200 * (y - x ** 2)))
make_convergence_plot(seq_nesterov, 2, 'x', ylabel = 'y')